<a href="https://colab.research.google.com/github/igor531205/Social_media/blob/main/CollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пушкарев Игорь Игоревич. Группа 23.М08-мм.
## Collaborative Filtering

1.   ✅ Разработать решение для рекомендации фильмов. Основная цель задания — разобраться в работе рекомендательных систем и метрик для их оценки. Исрользовать файл ratings.csv. Основная задача — построить коллаборативную рекомендательную систему (рекомендации строятся на основе взаимодействия пользователь-фильм) для предсказания рейтингов, которые пользователи ставят фильму. В качестве целевых метрик выступают RMSE и MAE, рассчитанные по первым 50 пользователям в файле ratings.csv (перед обучением модели первых 50 пользователей необходимо удалить из набора).

In [1]:
pip install scikit-surprise

In [2]:
from google.colab import files
from collections import defaultdict
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as warn
from surprise import Dataset, Reader, SVD, KNNBasic, accuracy
from surprise.model_selection import train_test_split

# для работы с графиками в интерактивном режиме
%matplotlib inline

# устанавливаем точность чисел с плавающей точкой
%precision %.4f

# отключаем предупреждения
warn.filterwarnings('ignore')

In [3]:
ratings_csv = 'ratings'
uploaded = files.upload_file(ratings_csv) # Загружаем файл ratings.csv

ratings_df = pd.read_csv(ratings_csv) # Считываем данные
ratings_df.head() # Выводим 5 строк

Saved ratings.csv to /content/ratings


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings_df.columns # Выведем все столбцы

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [5]:
ratings_df.shape # Размеры таблицы: (кол-во строк, кол-во столбцов)

(100004, 4)

In [6]:
ratings_df.describe() # Основная информация по количественным признакам

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [7]:
ratings_df.info() # Тип признаков и количество значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [8]:
ratings_df.nunique() # Проверяем количество уникальных значений

userId         671
movieId       9066
rating          10
timestamp    78141
dtype: int64

In [9]:
ratings_df.isnull().sum() # Проверяем общее количество пропущенных значений

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [10]:
# Перед обучением модели первых 50 пользователей необходимо удалить из набора
unique_users = ratings_df['userId'].unique()
train_users = unique_users[50:]
test_users = unique_users[:50]

# Разделение на обучающую и тестовую выборки (без колонки timestamp)
# рекомендации будут строиться на основе взаимодействия пользователь-фильм
train_df = ratings_df[ratings_df['userId'].isin(train_users)]
test_df = ratings_df[ratings_df['userId'].isin(test_users)]

print(f"Размер обучающей выборки: {train_df.shape} с количеством пользователей: {train_df['userId'].nunique()}")
print(f"Размер тестовой выборки: {test_df.shape} с количеством пользователей: {test_df['userId'].nunique()}")

Размер обучающей выборки: (91971, 4) с количеством пользователей: 621
Размер тестовой выборки: (8033, 4) с количеством пользователей: 50


In [11]:
# Reader используется для синтаксического анализа данных
reader = Reader(rating_scale=(0.5, 5))

# Загрузка набора данных из pandas dataframe
trainset = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader).build_full_trainset()
testset = [tuple(x) for x in test_df[['userId', 'movieId', 'rating']].values]

# Обучение модели SVD
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
svd.fit(trainset)

# Обучение модели KNN
knn = KNNBasic(name='cosine', user_based=True, random_state=42, verbose=False)
knn.fit(trainset)

# Предсказание рейтингов на тестовой выборке и вычисление RMSE и MAE для модели SVD
predictions_svd = svd.test(testset)

svd_rmse = accuracy.rmse(predictions_svd, verbose=False)
svd_mae = accuracy.mae(predictions_svd, verbose=False)

# Предсказание рейтингов на тестовой выборке и вычисление RMSE и MAE для модели KNN
predictions_knn = knn.test(testset)

knn_rmse = accuracy.rmse(predictions_knn, verbose=False)
knn_mae = accuracy.mae(predictions_knn, verbose=False)

print(f"Для 'Matrix Factorization-based algorithms' RMSE={svd_rmse:.02f} MAE={svd_mae:.02f}")
print(f"Для 'K-Nearest Neighbors algorithms' RMSE={knn_rmse:.02f} MAE={knn_mae:.02f}")

Для 'Matrix Factorization-based algorithms' RMSE=1.06 MAE=0.81
Для 'K-Nearest Neighbors algorithms' RMSE=1.11 MAE=0.88


In [12]:
# Коллаборативная рекомендательная система
def get_top_movieId(model_selection, userset, n=10):
    # Предсказание рекомендаций для пользователя
    predictions = model_selection.test(userset.to_records(index=False))

    # Собираем рекомендации
    user_ratings = [(pred.iid, pred.est) for pred in predictions]

    # Сортируем рекомендации по убыванию предсказанного рейтинга
    user_ratings.sort(key=lambda x: x[1], reverse=True)

    return user_ratings[:n]

# Рекомендации для 1 пользователя
userId = 1
userset = test_df[test_df['userId'] == userId][['userId', 'movieId', 'rating']]
recommendations = get_top_movieId(svd, userset)

# Вывод рекомендаций
print(f"Рекомендации для пользователя {userId}:")
print(*[f'{movieId=}, {predicted_rating=:.1f}' for movieId, predicted_rating in recommendations], sep = '\n')

Рекомендации для пользователя 1:
movieId=1172, predicted_rating=4.3
movieId=1953, predicted_rating=4.1
movieId=3671, predicted_rating=4.1
movieId=1293, predicted_rating=4.0
movieId=1287, predicted_rating=3.9
movieId=1263, predicted_rating=3.9
movieId=1343, predicted_rating=3.8
movieId=1029, predicted_rating=3.7
movieId=2968, predicted_rating=3.7
movieId=1061, predicted_rating=3.7
